In [1]:
import numpy as np
import opendssdirect as dss
import matplotlib.pyplot as plt
import pandas as pd
import re
import sys
from lib.compute_KCL_matrices import compute_KCL_matrices
from lib.basematrices import basematrices
from lib.NR3_timevarying import NR3_timevarying
from lib.network_mapper_OpenDSSnonvec import network_mapper_function as nmf2
from lib.NR3 import NR3_function

In [2]:
slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])
#fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
fn = 'IEEE_13Node_Modified_01.dss'
#fn = '02node_threephase_unbalanced_edit.dss'

times = np.linspace(0, 2*np.pi, 5)
der = [0, 0, 0, 0, 0]
capacitance = [0, 0, 0, 0, 0]

dss.run_command('Redirect ' + fn)
nnode = len(dss.Circuit.AllBusNames())
nline = len(dss.Lines.AllNames())

VNR01 = np.zeros((len(times), 3, nnode))
INR01=  np.zeros((len(times), 3, nline))
STXNR01 = np.zeros((len(times), 3, nline))
SRXNR01 = np.zeros((len(times), 3, nline))
iNR01 = np.zeros((len(times), 3, nnode))
sNR01 = np.zeros((len(times), 3, nnode))


X, g_SB, b_SB, G_KVL, b_KVL, H, g, b = basematrices(fn, slackidx, Vslack, None, None) #generate the all the matrices

for i in range(len(times)):
    VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
        NR3_timevarying(fn, X, g_SB, b_SB, G_KVL, b_KVL, H, g, b, None, None, der[i], capacitance[i], times[i]) 
    #run NR3 with variations in time, DER, and capacitance
    VNR01[i, :, :] = np.reshape(VNR, (3, nnode))
    INR01[i, :, :] = np.reshape(INR, (3, nline))
    STXNR01[i, :, :] = np.reshape(STXNR, (3, nline))
    SRXNR01[i, :, :] = np.reshape(SRXNR, (3, nline))
    iNR01[i, :, :] = np.reshape(iNR, (3, nnode))
    sNR01[i, :, :] = np.reshape(sNR, (3, nnode))


Iteration number 0.000000
Iteration number 1.000000
Iteration number 2.000000
Iteration number 3.000000
Iteration number 4.000000
VNR
[[ 1.        +0.j          0.91821246-0.10957476j  0.94941713-0.05560837j
   0.        +0.j          0.        +0.j          0.90898503-0.11336917j
   0.90169707-0.11722609j  0.        +0.j          0.91005304-0.10758234j
   0.94379241-0.07218327j  0.99789667-0.00254538j  0.95583011-0.05345307j
   0.91821246-0.10957476j  0.95262362-0.05453072j  0.91603083-0.10972829j]
 [-0.5       -0.8660254j  -0.53449743-0.85186265j -0.51752657-0.83897394j
  -0.51336593-0.83343754j -0.51205107-0.83131772j -0.53898377-0.85237005j
  -0.54286784-0.85266959j  0.        +0.j          0.        +0.j
  -0.52311502-0.84534436j -0.50086179-0.8649375j  -0.51809758-0.84317945j
  -0.53449743-0.85186265j -0.51781207-0.84107669j  0.        +0.j        ]
 [-0.5       +0.8660254j  -0.38105732+0.80661364j -0.42607817+0.830691j
  -0.42587663+0.83523106j -0.4241427 +0.83395432j -0.3800959

C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: ComplexWarning: Casting complex values to real discards the imaginary part


Iteration number 0.000000
Iteration number 1.000000
Iteration number 2.000000
Iteration number 3.000000
Iteration number 4.000000
VNR
[[ 1.        +0.j          0.91702735-0.11069987j  0.94871795-0.05618052j
   0.        +0.j          0.        +0.j          0.90767991-0.11450445j
   0.90029322-0.11837454j  0.        +0.j          0.90878072-0.10867913j
   0.94298277-0.07292763j  0.99786666-0.00257168j  0.95519986-0.05400536j
   0.91702735-0.11069987j  0.95195891-0.05509294j  0.91482195-0.11085709j]
 [-0.5       -0.8660254j  -0.53477608-0.85172867j -0.51766227-0.83871129j
  -0.51345692-0.83311909j -0.51212784-0.83097858j -0.53929195-0.85223708j
  -0.54319887-0.85253567j  0.        +0.j          0.        +0.j
  -0.52329715-0.8451453j  -0.50086855-0.86492693j -0.51823945-0.84295744j
  -0.53477608-0.85172867j -0.51795086-0.84083436j  0.        +0.j        ]
 [-0.5       +0.8660254j  -0.37968538+0.80546426j -0.42523648+0.83003544j
  -0.42503157+0.83462318j -0.42327961+0.83333112j -0.37872

Iteration number 0.000000
Iteration number 1.000000
Iteration number 2.000000
Iteration number 3.000000
Iteration number 4.000000
VNR
[[ 1.        +0.j          0.914686  -0.11291097j  0.9473375 -0.05730515j
   0.        +0.j          0.        +0.j          0.90510199-0.11673475j
   0.89752064-0.12063004j  0.        +0.j          0.90626787-0.11083371j
   0.94138336-0.07439079j  0.99780738-0.00262339j  0.953955  -0.05509109j
   0.914686  -0.11291097j  0.95064625-0.05619812j  0.91243381-0.11307525j]
 [-0.5       -0.8660254j  -0.53532559-0.85147318j -0.51792962-0.83820022j
  -0.51363636-0.8324985j  -0.51227932-0.83031746j -0.53989969-0.8519835j
  -0.54385166-0.85228017j  0.        +0.j          0.        +0.j
  -0.5236561 -0.84476004j -0.50088185-0.86490638j -0.51851891-0.84252596j
  -0.53532559-0.85147318j -0.51822426-0.84036309j  0.        +0.j        ]
 [-0.5       +0.8660254j  -0.37698227+0.80318913j -0.42357853+0.82873752j
  -0.42336695+0.83341903j -0.42157953+0.83209669j -0.376011

In [3]:
#fn= 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
fn = 'IEEE_13Node_Modified_01.dss'
#fn = '02node_threephase_unbalanced_edit.dss'

VNR02 = np.zeros((len(times), 3, nnode))
INR02=  np.zeros((len(times), 3, nline))
STXNR02 = np.zeros((len(times), 3, nline))
SRXNR02 = np.zeros((len(times), 3, nline))
iNR02 = np.zeros((len(times), 3, nnode))
sNR02 = np.zeros((len(times), 3, nnode))

for i in range(len(times)):
    network1 = nmf2(fn, times[i])
    #network1 = network_mapper_function('compare__lindist3flow_05node_threephase_unbalanced_oscillation_03.dss')

    nnode = network1.nodes.nnode
    nline = network1.lines.nline

    network1.cons.wpu = np.zeros((3,nnode))
    network1.vvc.vvcpu = np.zeros((3,nnode))

    network1.loads.aPQ =1*np.ones((3,nnode))*(network1.loads.spu != 0)
    network1.loads.aI = 0*np.ones((3,nnode))*(network1.loads.spu != 0)
    network1.loads.aZ = 0*np.ones((3,nnode))*(network1.loads.spu != 0)

    network1.loads.spu = 1*network1.loads.spu

    slackidx = 0
    Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])


    VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
        NR3_function(network1,slackidx,Vslack,None,None,None,None)

    VNR02[i, :, :] = np.reshape(VNR, (3, nnode))
    INR02[i, :, :] = np.reshape(INR, (3, nline))
    STXNR02[i, :, :] = np.reshape(STXNR, (3, nline))
    SRXNR02[i, :, :] = np.reshape(SRXNR, (3, nline))
    iNR02[i, :, :] = np.reshape(iNR, (3, nnode))
    sNR02[i, :, :] = np.reshape(sNR, (3, nnode))
    print('VNR\n', np.round(VNR,decimals=6))
    print('INR\n', np.round(INR,decimals=6))
    print('STXNR\n', np.round(STXNR,decimals=6))
    print('SRXNR\n', np.round(SRXNR,decimals=6))
    print('iNR\n', np.round(iNR,decimals=6))
    print('sNR\n', np.round(sNR,decimals=6))
    print('itercount:', itercount)

    print('|VNR|\n', np.round(np.abs(VNR),decimals=6))
    print('<VNR\n', np.round(180/np.pi*np.angle(VNR),decimals=6))
    print('D<VNR\n', 180/np.pi*np.angle(VNR) - 180/np.pi*np.angle(VNR[:,[0]]))

['sourcebus', '671', '634', '645', '646', '692', '675', '611', '652', '670', '650', '632', '680', '633', '684']
BASE
Vbase: 2401.7771198288433
Sbase: 1000000.0
Ibase: 416.358367204057
Zbase: 5.768533333333334

NODES
nnode: 15
nodelist: ['sourcebus', '671', '634', '645', '646', '692', '675', '611', '652', '670', '650', '632', '680', '633', '684']
phases: ['abc', 'abc', 'abc', 'bc', 'bc', 'abc', 'abc', 'c', 'a', 'abc', 'abc', 'abc', 'abc', 'abc', 'ac']
PH:
 [[1 1 1 0 0 1 1 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 0 0 1 1 1 1 1 0]
 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1]]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[0, 1, 1]
[0, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 0, 1]
[0, 0, 1]
[1, 0, 0]
LINES
nline: 14
TXnode: ['sourcebus', '650', '632', '670', '671', '632', '633', '632', '645', '671', '692', '671', '684', '684']
RXnode: ['650', '632', '670', '671', '680', '633', '634', '645', '646', '692', '675', '684', '611', '652']
TXnum: [ 0 10 11  9  1 11 13 11  3  1  5  1 14 14]
RXnum: [10 11

 [[ 41.38951814-38.50730018j   0.         +0.j
  -13.88905655 +5.76472733j]
 [  0.         +0.j           0.         +0.j
    0.         +0.j        ]
 [-13.88905655 +5.76472733j   0.         +0.j
   41.67719248-38.33148247j]]
12 - FYpu:
 [[ 0.         +0.j          0.         +0.j
   0.         +0.j        ]
 [ 0.         +0.j          0.         +0.j
   0.         +0.j        ]
 [ 0.         +0.j          0.         +0.j
  37.66843926-38.18704627j]]
13 - FYpu:
 [[24.75323379-9.44771471j  0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j        ]]

LOADS
aPQ:
 [[0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0.]]
aI:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: ComplexWarning: Casting complex values to real discards the imaginary part


NODES
nnode: 15
nodelist: ['sourcebus', '671', '634', '645', '646', '692', '675', '611', '652', '670', '650', '632', '680', '633', '684']
phases: ['abc', 'abc', 'abc', 'bc', 'bc', 'abc', 'abc', 'c', 'a', 'abc', 'abc', 'abc', 'abc', 'abc', 'ac']
PH:
 [[1 1 1 0 0 1 1 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 0 0 1 1 1 1 1 0]
 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1]]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[0, 1, 1]
[0, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 0, 1]
[0, 0, 1]
[1, 0, 0]
LINES
nline: 14
TXnode: ['sourcebus', '650', '632', '670', '671', '632', '633', '632', '645', '671', '692', '671', '684', '684']
RXnode: ['650', '632', '670', '671', '680', '633', '634', '645', '646', '692', '675', '684', '611', '652']
TXnum: [ 0 10 11  9  1 11 13 11  3  1  5  1 14 14]
RXnum: [10 11  9  1 12 13  2  3  4  5  6 14  7  8]
phases: ['abc', 'abc', 'abc', 'abc', 'abc', 'abc', 'abc', 'bc', 'bc', 'abc', 'abc', 'ac', 'c', 'a']
PH:
 [[1 1 1 1 1 1 1 0 0 1 1 1 0 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0]
 [1 1


LOADS
aPQ:
 [[0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0.]]
aI:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
aZ:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
ppu:
 [[0.    0.385 0.16  0.    0.    0.085 0.485 0.    0.128 0.017 0.    0.
  0.    0.    0.   ]
 [0.    0.385 0.12  0.17  0.115 0.    0.068 0.    0.    0.066 0.    0.
  0.    0.    0.   ]
 [0.    0.385 0.12  0.    0.115 0.085 0.29  0.17  0.    0.117 0.    0.
  0.    0.    0.   ]]
qpu:
 [[0.     0.22   0.11   0.     0.     0.0755 0.19   0.     0.086  0.01
  0.     0.     0.     0.     0.    ]
 [0.     0.22   0.09   0.125  0.066  0.     0.06   0.     0.     0.038
  0.     0.     0.     0.     0.    ]
 [0.     0.22   0.09   0.     0.066 

NODES
nnode: 15
nodelist: ['sourcebus', '671', '634', '645', '646', '692', '675', '611', '652', '670', '650', '632', '680', '633', '684']
phases: ['abc', 'abc', 'abc', 'bc', 'bc', 'abc', 'abc', 'c', 'a', 'abc', 'abc', 'abc', 'abc', 'abc', 'ac']
PH:
 [[1 1 1 0 0 1 1 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 0 0 1 1 1 1 1 0]
 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1]]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[0, 1, 1]
[0, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 0, 1]
[0, 0, 1]
[1, 0, 0]
LINES
nline: 14
TXnode: ['sourcebus', '650', '632', '670', '671', '632', '633', '632', '645', '671', '692', '671', '684', '684']
RXnode: ['650', '632', '670', '671', '680', '633', '634', '645', '646', '692', '675', '684', '611', '652']
TXnum: [ 0 10 11  9  1 11 13 11  3  1  5  1 14 14]
RXnum: [10 11  9  1 12 13  2  3  4  5  6 14  7  8]
phases: ['abc', 'abc', 'abc', 'abc', 'abc', 'abc', 'abc', 'bc', 'bc', 'abc', 'abc', 'ac', 'c', 'a']
PH:
 [[1 1 1 1 1 1 1 0 0 1 1 1 0 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0]
 [1 1

 [[0.  0.  0.  0.  0.  0.  0.2 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.2 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.2 0.1 0.  0.  0.  0.  0.  0.  0. ]]

CONS
wpu:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
VNR
 [[ 1.      +0.j        0.915848-0.111816j  0.948022-0.056748j
   0.      +0.j        0.      +0.j        0.906381-0.11563j
   0.898896-0.119513j  0.      +0.j        0.907515-0.109766j
   0.942177-0.073666j  0.997837-0.002598j  0.954573-0.054553j
   0.915848-0.111816j  0.951298-0.055651j  0.913619-0.111976j]
 [-0.5     -0.866025j -0.535053-0.851598j -0.517797-0.838453j
  -0.513547-0.832805j -0.512204-0.830644j -0.539598-0.852108j
  -0.543528-0.852405j  0.      +0.j        0.      +0.j
  -0.523478-0.84495j  -0.500875-0.864917j -0.51838 -0.842739j
  -0.535053-0.851598j -0.518089-0.840596j  0.      +0.j      ]
 [-0.5     +0.866025j -0.378323+


LOADS
aPQ:
 [[0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0.]]
aI:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
aZ:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
ppu:
 [[0.    0.385 0.16  0.    0.    0.085 0.485 0.    0.128 0.017 0.    0.
  0.    0.    0.   ]
 [0.    0.385 0.12  0.17  0.115 0.    0.068 0.    0.    0.066 0.    0.
  0.    0.    0.   ]
 [0.    0.385 0.12  0.    0.115 0.085 0.29  0.17  0.    0.117 0.    0.
  0.    0.    0.   ]]
qpu:
 [[0.     0.22   0.11   0.     0.     0.0755 0.19   0.     0.086  0.01
  0.     0.     0.     0.     0.    ]
 [0.     0.22   0.09   0.125  0.066  0.     0.06   0.     0.     0.038
  0.     0.     0.     0.     0.    ]
 [0.     0.22   0.09   0.     0.066 

phases: ['abc', 'abc', 'abc', 'bc', 'bc', 'abc', 'abc', 'c', 'a', 'abc', 'abc', 'abc', 'abc', 'abc', 'ac']
PH:
 [[1 1 1 0 0 1 1 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 0 0 1 1 1 1 1 0]
 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1]]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[0, 1, 1]
[0, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 0, 1]
[0, 0, 1]
[1, 0, 0]
LINES
nline: 14
TXnode: ['sourcebus', '650', '632', '670', '671', '632', '633', '632', '645', '671', '692', '671', '684', '684']
RXnode: ['650', '632', '670', '671', '680', '633', '634', '645', '646', '692', '675', '684', '611', '652']
TXnum: [ 0 10 11  9  1 11 13 11  3  1  5  1 14 14]
RXnum: [10 11  9  1 12 13  2  3  4  5  6 14  7  8]
phases: ['abc', 'abc', 'abc', 'abc', 'abc', 'abc', 'abc', 'bc', 'bc', 'abc', 'abc', 'ac', 'c', 'a']
PH:
 [[1 1 1 1 1 1 1 0 0 1 1 1 0 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 0]]
config: ['mtx601', 'mtx601', 'mtx601', 'mtx601', 'mtx601', 'mtx602', 'mtx602', 'mtx603', 'mtx603', 'mtx606', 'm

VNR
 [[ 1.      +0.j        0.913553-0.113975j  0.94667 -0.057847j
   0.      +0.j        0.      +0.j        0.903855-0.117808j
   0.89618 -0.121715j  0.      +0.j        0.905052-0.11187j
   0.94061 -0.075095j  0.997779-0.002648j  0.953353-0.055614j
   0.913553-0.113975j  0.950011-0.05673j   0.911278-0.114143j]
 [-0.5     -0.866025j -0.535591-0.851354j -0.518059-0.837957j
  -0.513723-0.832202j -0.512353-0.830002j -0.540193-0.851865j
  -0.544167-0.852161j  0.      +0.j        0.      +0.j
  -0.523829-0.844577j -0.500888-0.864897j -0.518654-0.842321j
  -0.535591-0.851354j -0.518356-0.840139j  0.      +0.j      ]
 [-0.5     +0.866025j -0.375678+0.802086j -0.422779+0.828108j
  -0.422564+0.832835j -0.420759+0.831498j -0.374704+0.798527j
  -0.373795+0.796428j -0.369197+0.799859j  0.      +0.j
  -0.406896+0.822313j -0.496459+0.864463j -0.425644+0.833213j
  -0.375678+0.802086j -0.424211+0.83066j  -0.372833+0.800789j]]
INR
 [[ 1.355276-0.728251j  1.355276-0.728251j  1.187761-0.597349j
   1.16

In [4]:
VDSS0 = np.zeros((len(times), 3, nnode))
IDSS0 =  np.zeros((len(times), 3, nline))
SRXDSS0 = np.zeros((len(times), 3, nline))
STXDSS0 = np.zeros((len(times), 3, nline))


for i in range(len(times)):
    Vbase = 2771.2812921102036
    Sbase = 1000000.0
    Ibase = Sbase/Vbase
    Zbase = Vbase/Ibase

    #dss.run_command('Redirect compare_opendss_05node_threephase_unbalanced_oscillation_03.dss') #Using OpenDSS
    dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
    #dss.run_command('Redirect 02node_threephase_unbalanced_edit.dss')
    # Set slack bus (sourcebus) voltage reference in p.u.
    SlackBusVoltage = 1.000
    dss.Vsources.PU(SlackBusVoltage)
    
    #Time varying load
    for k in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[k])

        dss.Loads.kW(dss.Loads.kW() * (1 + 0.1*np.sin(2*np.pi*0.01*times[i])) )
        dss.Loads.kvar(dss.Loads.kvar() * (1 + 0.1*np.sin(2*np.pi*0.01*times[i])))
      
   
    dss.Solution.Convergence(0.000000000001)
    
   

    # Solve power flow with OpenDSS file
    dss.Solution.Solve()
    if not dss.Solution.Converged:
        print('Initial Solution Not Converged. Check Model for Convergence')
    else:
        print('Initial Model Converged. Proceeding to Next Step.')
        #Doing this solve command is required for GridPV, that is why the monitors
        #go under a reset process
        dss.Monitors.ResetAll()

        #set solution Params
        #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
        dss.Solution.Mode(1)
        dss.Solution.Number(1)
        dss.Solution.StepSize(1)
        dss.Solution.ControlMode(-1)
        dss.Solution.MaxControlIterations(1000000)
        dss.Solution.MaxIterations(30000)
        #Easy process to get all names and count of loads, a trick to avoid
        #some more lines of code
        TotalLoads=dss.Loads.Count()
        AllLoadNames=dss.Loads.AllNames()
        print('OpenDSS Model Compliation Done.')
        print('Iterations: ', dss.Solution.Iterations())
        print('Tolerance: ', dss.Solution.Convergence())

    print('')


#     for k in range(len(dss.Loads.AllNames())):
#         dss.Loads.Name(dss.Loads.AllNames()[k])
     
#         print("\n")
#         print(dss.Loads.kW() * (1 + 0.1*np.sin(2*np.pi*0.01*times[i])))
#         print((dss.Loads.kvar() * (1 + 0.1*np.sin(2*np.pi*0.01*times[i]))))
      
   
    # print(dss.Solution.Converged())

    # Print number of buses, and bus names
    print(len(dss.Circuit.AllBusNames()))
    print(dss.Circuit.AllBusNames())

    # Print number of loads, and load names
    print(len(dss.Loads.AllNames()))
    print(dss.Loads.AllNames())

    print('')

    VDSS = np.zeros((3,len(dss.Circuit.AllBusNames())),dtype='complex')

    for k1 in range(len(dss.Circuit.AllBusNames())):
        
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
        
    #     print(dss.Circuit.AllBusNames()[k1])
    #     print(dss.Bus.Nodes())

    #     print('puVOTLAGES - LN CARTESIAN')
    #     print(dss.Bus.PuVoltage())

        ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1

        Vtemp = np.asarray(dss.Bus.PuVoltage())
        Vtemp = Vtemp[0:5:2] + 1j*Vtemp[1:6:2]

    #     print(np.asarray(dss.Bus.Nodes(),'int'))

        VDSS[ph,k1] = Vtemp    

    #     VDSS[np.asarray(dss.Bus.Nodes(),'int'),k1] = np.array(dss.Bus.PuVoltage()[0:5:2] + 1j*dss.Bus.PuVoltage()[1:6:2])


    #     VDSS[dss.Bus.Nodes()-1,k1] = dss.Bus.PuVoltage()[0:2:5]
    #     for k2 in range(len(dss.Bus.Nodes())):
    #         VDSS[int(dss.Bus.Nodes()[k2])-1,k1] = dss.Bus.PuVoltage()[2*k2] + 1j*dss.Bus.PuVoltage()[2*k2+1]

    print('VDSS\n', np.round(VDSS,decimals=6))


    IDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

    for k1 in range(len(dss.Lines.AllNames())):
        dss.Lines.Name(dss.Lines.AllNames()[k1])
    #     print(dss.Lines.AllNames()[k1])
        ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
        Imn = np.asarray(dss.CktElement.Currents())/Ibase
    #     print(Imn)
        Imn = Imn[0:int(len(Imn)/2)]
    #     print(Imn)
        Imn = Imn[0:5:2] + 1j*Imn[1:6:2]
    #     print(Imn)
        IDSS[ph,k1] = Imn
    #     print('')

    print('IDSS\n', np.round(IDSS,decimals=6))

    STXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')
    SRXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

    for k1 in range(len(dss.Lines.AllNames())):
        dss.Lines.Name(dss.Lines.AllNames()[k1])
    #     print(dss.Lines.AllNames()[k1])
    #     print(dss.CktElement.BusNames())
        ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
    #     print(ph)
        Sk = np.asarray(dss.CktElement.Powers())/(Sbase/1000)
    #     print(Sk)

    #     print(Sk[0:int(len(Sk)/2)])
    #     print(Sk[int(len(Sk)/2):])

        STXtemp = Sk[0:int(len(Sk)/2)]
        SRXtemp = Sk[int(len(Sk)/2):]

        STXtemp = STXtemp[0:5:2] + 1j*STXtemp[1:6:2]
    #     print(STXtemp)

        SRXtemp = -(SRXtemp[0:5:2] + 1j*SRXtemp[1:6:2])
    #     print(SRXtemp)

        STXDSS[ph,k1] = STXtemp
        SRXDSS[ph,k1] = SRXtemp

    VDSS0[i, :, :] = VDSS
    IDSS0[i, :, :] = IDSS
    SRXDSS0[i, :, :] = SRXDSS
    STXDSS0[i, :, :] = STXDSS
    
    
    print('STXDSS\n', np.round(STXDSS,decimals=6))
    print('SRXDSS\n', np.round(SRXDSS,decimals=6))

    print('|VDSS|\n', np.round(np.abs(VDSS),decimals=6))
    print('<VDSS\n', np.round(180/np.pi*np.angle(VDSS),decimals=6))
    print('D<VDSS\n', 180/np.pi*np.angle(VDSS) - 180/np.pi*np.angle(VDSS[:,[0]]))

Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.
Iterations:  9
Tolerance:  1e-12

15
['sourcebus', '671', '634', '645', '646', '692', '675', '611', '652', '670', '650', '632', '680', '633', '684']
15
['671', '634a', '634b', '634c', '645', '646', '692', '675a', '675b', '675c', '611', '652', '670a', '670b', '670c']

VDSS
 [[ 0.999808-2.31000e-04j  0.878117-1.33969e-01j  0.931033-6.53090e-02j
   0.      +0.00000e+00j  0.      +0.00000e+00j  0.868665-1.39676e-01j
   0.863452-1.45725e-01j  0.      +0.00000e+00j  0.870902-1.32021e-01j
   0.918081-8.68860e-02j  0.996836-3.23700e-03j  0.93739 -6.33560e-02j
   0.878117-1.33969e-01j  0.934211-6.43320e-02j  0.876138-1.34097e-01j]
 [-0.50012 -8.65773e-01j -0.581687-7.94743e-01j -0.539743-8.02596e-01j
  -0.531896-7.91948e-01j -0.528455-7.86690e-01j -0.59088 -7.94882e-01j
  -0.598735-7.94674e-01j  0.      +0.00000e+00j  0.      +0.00000e+00j
  -0.553811-8.02101e-01j -0.502044-8.62972e-01j -0.540518-8.06948e-01j
  -0

VDSS
 [[ 0.999801-2.33000e-04j  0.873897-1.34205e-01j  0.928749-6.52370e-02j
   0.      +0.00000e+00j  0.      +0.00000e+00j  0.864201-1.39793e-01j
   0.858839-1.45778e-01j  0.      +0.00000e+00j  0.86655 -1.32153e-01j
   0.91527 -8.69350e-02j  0.996728-3.23600e-03j  0.93527 -6.33110e-02j
   0.873897-1.34205e-01j  0.932009-6.42740e-02j  0.871871-1.34324e-01j]
 [-0.500118-8.65765e-01j -0.581337-7.91267e-01j -0.539236-8.00092e-01j
  -0.53111 -7.89288e-01j -0.527545-7.83949e-01j -0.590561-7.91364e-01j
  -0.598412-7.91121e-01j  0.      +0.00000e+00j  0.      +0.00000e+00j
  -0.553392-7.99341e-01j -0.502021-8.62852e-01j -0.540088-8.04576e-01j
  -0.581337-7.91267e-01j -0.539662-8.02334e-01j  0.      +0.00000e+00j]
 [-0.499671+8.65967e-01j -0.333424+7.99931e-01j -0.395689+8.27468e-01j
  -0.391247+8.29040e-01j -0.387204+8.25982e-01j -0.332039+7.98754e-01j
  -0.330577+8.00547e-01j -0.328359+7.97591e-01j  0.      +0.00000e+00j
  -0.374707+8.20984e-01j -0.494836+8.64362e-01j -0.398151+8.32261e-01

C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:156: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:157: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:158: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\kathl\Anaconda3\lib\site-packages\ipykernel_launcher.py:159: ComplexWarning: Casting complex values to real discards the imaginary part


In [16]:
dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
cap_dict = {}
cap_ph_dict = {}
for n in range(len(dss.Capacitors.AllNames())):
    dss.Capacitors.Name(dss.Capacitors.AllNames()[n])
    #print(dss.CktElement.BusNames()[0])
    pattern =  r"(\w+)\."  
    m = re.findall(pattern, dss.CktElement.BusNames()[0])
    cap_dict[m[0]] = dss.Capacitors.kvar()    
    load_phases = [0, 0, 0]
    for i in range(1, 4): #if the phase is present, what other phases are
        pattern = r"\.%s" % (str(i))
        m2 = re.findall(pattern, dss.CktElement.BusNames()[0])
        if m2:
            load_phases[i - 1] = 1
    cap_ph_dict[m[0]] = load_phases
    

for k2 in range(1, len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[k2])
    lc = dss.Lines.LineCode()
    dss.LineCodes.Name(lc)
    print(dss.LineCodes.Xmatrix())

[1.0179, 0.5017, 0.4236, 0.5017, 1.0478, 0.3849, 0.4236, 0.3849, 1.0348]
[1.0179, 0.5017, 0.4236, 0.5017, 1.0478, 0.3849, 0.4236, 0.3849, 1.0348]
[1.0179, 0.5017, 0.4236, 0.5017, 1.0478, 0.3849, 0.4236, 0.3849, 1.0348]
[1.0179, 0.5017, 0.4236, 0.5017, 1.0478, 0.3849, 0.4236, 0.3849, 1.0348]
[1.1814, 0.4236, 0.5017, 0.4236, 1.1983, 0.3849, 0.5017, 0.3849, 1.2112]
[1.1814, 0.4236, 0.5017, 0.4236, 1.1983, 0.3849, 0.5017, 0.3849, 1.2112]
[1.3569, 0.4591, 0.4591, 1.3471]
[1.3569, 0.4591, 0.4591, 1.3471]
[0.438352, 0.0276838, -0.0184204, 0.0276838, 0.396697, 0.0276838, -0.0184204, 0.0276838, 0.438352]
[0.438352, 0.0276838, -0.0184204, 0.0276838, 0.396697, 0.0276838, -0.0184204, 0.0276838, 0.438352]
[1.3569, 0.4591, 0.4591, 1.3471]
[1.3475]
[0.5124]


In [6]:
# INR01[0].imag-IDSS[0].imag


In [7]:
#dss - opendss solution
#01 - vectorized solution without network mapper
#02 - nonvectorized solution using OpenDSS and network mapper
for i in range(len(times)):
    print(i)
    print('Complex Voltage difference')
    print(np.max(np.abs(VNR01[i] - VNR02[i])))
    print(np.max(np.abs(VDSS0[i] - VNR01[i]))) 
    print(np.max(np.abs(VDSS0[i] - VNR02[i])))

    print('Complex Current difference')
    print(np.max(np.abs(INR01[i] - INR02[i])))
    print(np.max(np.abs(IDSS0[i] - INR01[i])))
    print(np.max(np.abs(IDSS0[i] - INR02[i])))

    print('Complex TX Power difference')
    print(np.max(np.abs(STXNR01[i] - STXNR02[i])))
    print(np.max(np.abs(STXDSS0[i] - STXNR01[i])))
    print(np.max(np.abs(STXDSS0[i] - STXNR02[i])))

    print('Complex RX Power difference')
    print(np.max(np.abs(SRXNR01[i] - SRXNR02[i])))
    print(np.max(np.abs(SRXDSS0[i] - SRXNR01[i])))
    print(np.max(np.abs(SRXDSS0[i] - SRXNR02[i])))

    print('Voltage Magnitude difference')
    print(np.max(np.abs(np.abs(VNR01[i]) - np.abs(VNR02[i]))))
    print(np.max(np.abs(np.abs(VDSS0[i]) - np.abs(VNR01[i]))))
    print(np.max(np.abs(np.abs(VDSS0[i]) - np.abs(VNR02[i]))))

    print('Voltage Angle difference')
    print(np.max(np.abs(180/np.pi*np.angle(VNR01[i]) - 180/np.pi*np.angle(VNR02[i]))))
    print(np.max(np.abs(180/np.pi*np.angle(VDSS0[i]) - 180/np.pi*np.angle(VNR01[i]))))
    print(np.max(np.abs(180/np.pi*np.angle(VDSS0[i]) - 180/np.pi*np.angle(VNR02[i]))))
    print("\n")

0
Complex Voltage difference
1.1102230246251565e-16
0.05586681963504425
0.05586681963504436
Complex Current difference
2.220446049250313e-16
0.8130495669495529
0.8130495669495529
Complex TX Power difference
2.220446049250313e-16
0.8123880305659268
0.8123880305659266
Complex RX Power difference
2.220446049250313e-16
0.8117598412668653
0.8117598412668652
Voltage Magnitude difference
1.1102230246251565e-16
0.05586681963504425
0.05586681963504436
Voltage Angle difference
0.0
0.0
0.0


1
Complex Voltage difference
2.220446049250313e-16
0.05538227509724125
0.05538227509724114
Complex Current difference
2.220446049250313e-16
0.8349857782231398
0.8349857782231398
Complex TX Power difference
2.220446049250313e-16
0.816962303971847
0.816962303971847
Complex RX Power difference
2.220446049250313e-16
0.8163047774389821
0.8163047774389821
Voltage Magnitude difference
2.220446049250313e-16
0.05538227509724125
0.05538227509724114
Voltage Angle difference
0.0
0.0
0.0


2
Complex Voltage difference
1.6

In [8]:
0 #without capacitors
Complex Voltage difference
1.0
0.049097506284065284
0.9997491664218399
Complex Current difference
1.296840007074481
1.112273503547586
2.1209049614799382
Complex TX Power difference
1.420140049747832
0.7018104665006095
1.8505036439566709
Complex RX Power difference
1.4166120626846108
0.7007257087925388
1.8459096867475413
Voltage Magnitude difference
1.0
0.049097506284065284
0.9997491664218399
Voltage Angle difference
180.0
0.0
180.0


1
Complex Voltage difference
1.0
0.04983645366418754
0.9997456861840414
Complex Current difference
1.3099695675830405
1.130692235774802
2.1494412723199336
Complex TX Power difference
1.4361216436387785
0.7050016144260078
1.8624921203367983
Complex RX Power difference
1.4325078897755275
0.7038836813225384
1.8578113116604063
Voltage Magnitude difference
1.0
0.04983645366418754
0.9997456861840414
Voltage Angle difference
180.0
0.0
180.0


2
Complex Voltage difference
1.0
0.05056371739529775
0.9997422313678557
Complex Current difference
1.322975978382606
1.1490130186799752
2.1777813344422396
Complex TX Power difference
1.4520057032192064
0.7080716524362469
1.874248929379419
Complex RX Power difference
1.4483055241342655
0.7069201514799779
1.8694813731363829
Voltage Magnitude difference
1.0
0.05056371739529775
0.9997422313678557
Voltage Angle difference
180.0
0.0
180.0


3
Complex Voltage difference
1.0
0.0512722552163235
0.9997388361870655
Complex Current difference
1.3357322700771437
1.1670523950185219
2.2056440122751497
Complex TX Power difference
1.4676360373213533
0.7109946229464486
1.8856652777533522
Complex RX Power difference
1.4638496566238548
0.7098095119798243
1.8808119567390913
Voltage Magnitude difference
1.0
0.0512722552163235
0.9997388361870655
Voltage Angle difference
180.0
0.0
180.0


4
Complex Voltage difference
1.0
0.051955356970542765
0.9997355344708205
Complex Current difference
1.3481137975164157
1.184627674545338
2.2327508584022433
Complex TX Power difference
1.482857231156727
0.7137484728932857
1.8966387409783165
Complex RX Power difference
1.4789857959053643
0.7125300811475036
1.8917015272639386
Voltage Magnitude difference
1.0
0.051955356970542765
0.9997355344708205
Voltage Angle difference
180.0
0.0
180.0


SyntaxError: invalid syntax (<ipython-input-8-a8f34cecd078>, line 2)

In [ ]:
VNR01[0]

In [ ]:
VNR02[0]

In [ ]:
VDSS0[0]

In [ ]:
x = np.array([[1, 2], [4, 4]])
y = np.array([[3, 3], [3, 5]])
print(np.multiply(x, y))